In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [ ]:
# image transformations applied to dataset before feeding into the model (data augmentation)
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
# loading CIFAR-10 dataset
training_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testing_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

training_data_loader = DataLoader(dataset=training_dataset, batch_size=64, shuffle=True)
testing_data_loader = DataLoader(dataset=testing_dataset, batch_size=64, shuffle=True)

# CIFAR-10 dataset class labels
class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

In [ ]:
# define a CNN model using dropout to prevent overfitting
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.pool(self.bn1(nn.ReLU()(self.conv1(x))))
        x = self.pool(self.bn2(nn.ReLU()(self.conv2(x))))
        x = self.pool(self.bn3(nn.ReLU()(self.conv3(x))))
        x = torch.flatten(x, 1)
        x = self.dropout(nn.ReLU()(self.fc1(x)))
        x = self.dropout(nn.ReLU()(self.fc2(x)))
        x = self.fc3(x)
        return x

In [ ]:
# initialise the model, using a learning rate of 0.001
model = CNN()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)
loss_function = nn.CrossEntropyLoss()


In [ ]:
# train model for 50 epochs
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for i, data in enumerate(training_data_loader):
        images, labels = data

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # adjusts the learning rate during training
    scheduler.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss/len(training_data_loader):.4f}')

In [ ]:
# save the trained model
torch.save(model.state_dict(), "trained_model.pth")

In [ ]:
# load the model to evaluate on the testing data
model = CNN()
model.load_state_dict(torch.load("trained_model.pth"))

In [ ]:
# evaluate model
model.eval()
success = 0
total = 0

with torch.no_grad():
    for data in testing_data_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        success += (predicted == labels).sum().item()

test_accuracy = 100 * success / total
print(f'Test Accuracy: {test_accuracy:.2f}%')